<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/classify_category/classify_news_category_kobert_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [5]:
# text의 길이가 80이상 256이하의 문장 만들기
def text_splicing(text, max_seq_len) :
  text = text.split('\n')
  saved_text, s, i, data = '', 0, 0, []
  for t in text :
    if len(t) < 8 or t[0] in ['o','『', '(', '┌','│', '└', 'ㄴ', '┌','├','◎', '[', '■', 'ㄱ', '-', '.', '<'] : 
      continue
    if t[-1] in ['쪽', '-', '>', ']'] or t[1] in [')'] or t[0:2] in ['vs', '만,', '해!'] :
      continue
    if len(t) > max_seq_len :
        data.append(saved_text)
        data.append(t)
        saved_text, s = '', 0
        i += 1
    elif s + len(t) > max_seq_len :
        data.append(saved_text)
        saved_text, s = t, len(t)
        i += 1
    else :
        saved_text += t
        s += len(t)
  data.append(saved_text)
  
  ret_data = []
  for d in data :
    if len(d) >= 80 :
      ret_data.append(d)
  return ret_data

In [6]:
import os
max_seq_len = 256
dataset_train = []
labels = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              # labels.append( path[path.rindex('/')+1:])
              # dataset_train.append(f.read()[:max_seq_len])
              
              label = path[path.rindex('/')+1:]
              text = f.read()
              data = text_splicing(text, max_seq_len)
              dataset_train.extend(data)
              labels.extend([label]*len(data))

len(dataset_train), len(labels), set(labels)

(137023,
 137023,
 {'ITscience',
  'culture',
  'economy',
  'entertainment',
  'health',
  'life',
  'politic',
  'social',
  'sport'})

## 전처리

In [7]:
import pandas as pd

# labels_classes
df = pd.DataFrame({'content' : dataset_train, 'label' : labels})

# dataset의 balance 여부 확인
df.groupby(by=['label']).count()

,content
label,
ITscience,16362
culture,15139
economy,16302
entertainment,15334
health,16004
life,14764
politic,13766
social,14396
sport,14956


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df['label'])
num_labels = len(label_encoder.classes_)

df['encoded_label'] = np.asarray(label_encoder.transform(df['label']), dtype=np.int32)
df.tail(3)
# label_encoder.inverse_transform(prediction_test)

,content,label,encoded_label
137020,된장국도 끓이고 배추 겉저리도 담궜다. 두루 둘러앉아 식사를 하는데 큰 형님 게오르...,life,5
137021,그리고 마무리는 게오르기 형님이 낮에 준비한 커다란 수박을 잘라먹었다.수박을 커다란...,life,5
137022,그리고는 내게 이렇게 오셔서 덕분에 우리가 고려 말을 많이 합니다.그리고 앞으로도 ...,life,5


In [9]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, test_texts, train_labels, test_labels = train_test_split(df.content, df.encoded_label, test_size=0.2, random_state=123, shuffle=True, stratify=df.encoded_label)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=123, shuffle=True, stratify=train_labels)

print(len(train_texts), len(train_labels))
print(len(val_texts), len(val_labels))
print(len(test_texts), len(test_labels))

87694 87694
21924 21924
27405 27405


In [10]:
dataset_train, dataset_val, dataset_test = [], [], []
text, label = '', ''
for text, label in zip(train_texts, train_labels) :
  dataset_train.append([text, label])
for text, label in zip(val_texts, val_labels) :
  dataset_val.append([text, label])
for text, label in zip(test_texts, test_labels) :
  dataset_test.append([text, label])
len(dataset_train), len(dataset_val), len(dataset_test)

(87694, 21924, 27405)

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
## Setting parameters
max_len = 256
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200

In [13]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1)

In [15]:
del labels

del df
del train_texts
del train_labels
del val_texts
del val_labels
del test_texts
del test_labels

del data_train
del data_val
del data_test
del tok
del tokenizer
del dataset_train
del dataset_val
del dataset_test

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768,
                 num_classes=num_labels, ## 분류할 클래스 수
                 dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [19]:
def train_for_epochs(num_epochs, train_dataloader, val_dataloader) :
  # Prepare optimizer and schedule (linear warmup and decay)
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)  # lr = learning rate
  loss_fn = nn.CrossEntropyLoss()

  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  
  # train
  for e in range(num_epochs):
      train_acc = 0.0
      val_acc = 0.0
      val_loss = 0.0
      # training
      model.train()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
          optimizer.zero_grad()
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          loss = loss_fn(out, label)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          train_acc += calc_accuracy(out, label)
          if batch_id % log_interval == 0:
              print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
      model.eval()
      # validation
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          val_loss += loss_fn(out, label)

          val_acc += calc_accuracy(out, label)
      print("epoch {} validation acc {} loss {}".format(e+1, val_acc / (batch_id+1), val_loss.data.cpu().numpy()))

In [22]:
def test(test_dataloader) :
  out_li = []
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)  # lr = learning rate
  loss_fn = nn.CrossEntropyLoss()

  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)
  test_acc, test_loss = 0.0, 0.0
  model.eval()

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    out_li.append(out)
    test_acc += calc_accuracy(out, label)
    test_loss += loss_fn(out, label)
  print("Test acc {}, loss {}".format(test_acc / (batch_id+1), test_loss))
  return out_li

In [28]:
import gc
gc.collect()
# torch.cuda.empty_cache()

476

In [21]:
train_for_epochs(num_epochs, train_dataloader, val_dataloader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5481 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.239077568054199 train acc 0.1875
epoch 1 batch id 201 loss 2.2967329025268555 train acc 0.12468905472636815
epoch 1 batch id 401 loss 1.9808170795440674 train acc 0.1561720698254364
epoch 1 batch id 601 loss 1.3976389169692993 train acc 0.25946339434276205
epoch 1 batch id 801 loss 1.4425305128097534 train acc 0.3578339575530587
epoch 1 batch id 1001 loss 1.2798954248428345 train acc 0.4288836163836164
epoch 1 batch id 1201 loss 0.8035870790481567 train acc 0.4777268942547877
epoch 1 batch id 1401 loss 0.6184626817703247 train acc 0.5149892933618844
epoch 1 batch id 1601 loss 0.6582783460617065 train acc 0.5441130543410369
epoch 1 batch id 1801 loss 0.8104749321937561 train acc 0.5659355913381455
epoch 1 batch id 2001 loss 0.6125224232673645 train acc 0.5858633183408296
epoch 1 batch id 2201 loss 0.3132786452770233 train acc 0.6002953203089505
epoch 1 batch id 2401 loss 0.8216688632965088 train acc 0.6132340691378593
epoch 1 batch id 2601 loss 0.27481183409690

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1371 [00:00<?, ?it/s]

...

epoch 9 validation acc 0.8425419401896426 loss 0.0003792193892877549


  0%|          | 0/5481 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0009809585753828287 train acc 1.0
epoch 10 batch id 201 loss 0.0005467613809742033 train acc 0.9934701492537313
epoch 10 batch id 401 loss 0.0005796737968921661 train acc 0.9932980049875312
epoch 10 batch id 601 loss 0.00042638438753783703 train acc 0.9931364392678869
epoch 10 batch id 801 loss 0.0007492119329981506 train acc 0.9925873907615481
epoch 10 batch id 1001 loss 0.01030302420258522 train acc 0.9927572427572428
epoch 10 batch id 1201 loss 0.0004734884132631123 train acc 0.992714404662781
epoch 10 batch id 1401 loss 0.0008631906239315867 train acc 0.9928622412562456
epoch 10 batch id 1601 loss 0.0036301936488598585 train acc 0.99289506558401
epoch 10 batch id 1801 loss 0.009461541660130024 train acc 0.9929553026096613
epoch 10 batch id 2001 loss 0.18386955559253693 train acc 0.9927848575712144
epoch 10 batch id 2201 loss 0.00038276176201179624 train acc 0.9929009541117674
epoch 10 batch id 2401 loss 0.0005393975297920406 train acc 0.9929196168263223
e

  0%|          | 0/1371 [00:00<?, ?it/s]

epoch 10 validation acc 0.8440007293946025 loss 0.0003491892130114138


In [26]:
torch.save(model, data_path+'/classify_category/epoch10_val_acc:0.844.pt')